In [1]:
import os
import sys
import gc

import scipy.stats as stats
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import AnnData
from fast_matrix_market import mmread

import scipy
from scipy.spatial.distance import jensenshannon
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams

import torch
import cell2location

# Set matplotlib parameters
rcParams['pdf.fonttype'] = 42  # Enables correct plotting of text for PDFs

/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated.

In [2]:
results_folder = '/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/cell2location/mixed'

# create paths and names to results folders for reference regression and cell2location models
ref_run_name = '/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/cell2location/reference_signatures'
run_name = f'{results_folder}/cell2location_map'

adata_file = f"{ref_run_name}/reference_major.h5ad"
adata_ref = sc.read_h5ad(adata_file)
mod = cell2location.models.RegressionModel.load(f"{ref_run_name}", adata_ref)

INFO     File                                                                                                      
         /bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/cell2location/referenc
         e_signatures/model.pt already downloaded                                                                  


/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/model/base/_utils.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = tor

Epoch 1/523:   0%|          | 1/523 [00:01<13:44,  1.58s/it, v_num=1]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/523:   0%|          | 1/523 [00:01<15:15,  1.75s/it, v_num=1]


In [ ]:
# Directories and paths
input_folder  = "/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/replicates/mixed/h5ad_objects"
output_folder = "/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/cell2location/mixed"

# Prepare reference signatures
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[
        f'means_per_cluster_mu_fg_{i}' for i in adata_ref.uns['mod']['factor_names']
    ]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}' for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']

# Iterate over all replicates
replicates = [f for f in os.listdir(input_folder) if f.endswith("GEX.h5ad")]

for replicate_name in replicates:
    replicate_path = os.path.join(input_folder, replicate_name)
    run_name = os.path.join(output_folder, f"cell2location_map_{replicate_name.split('_')[2]}")

    # Load Visium query dataset (replicate)
    adata_vis_0 = sc.read_h5ad(replicate_path)

    # Prepare `adata_vis_0` for cell2location
    adata_vis_0.X_norm = adata_vis_0.X
    adata_vis_0.X = np.expm1(adata_vis_0.X_norm).round()

    # Find shared genes and subset
    intersect = np.intersect1d(adata_vis_0.var_names, inf_aver.index)
    adata_vis_0 = adata_vis_0[:, adata_vis_0.var_names.isin(intersect)].copy()
    inf_aver = inf_aver.loc[inf_aver.index.isin(intersect), :].copy()

    # Ensure matching gene order
    adata_vis_0.var_names = adata_vis_0.var_names.sort_values()
    inf_aver = inf_aver.sort_index()

    # Setup cell2location
    cell2location.models.Cell2location.setup_anndata(adata=adata_vis_0)

    # Create and train the model
    mod = cell2location.models.Cell2location(
        adata_vis_0,
        cell_state_df=inf_aver,
        N_cells_per_location=5,
        detection_alpha=200,
    )
    mod.train(max_epochs=30000, batch_size=None, train_size=1, use_gpu=True)

    # Export posterior and save results
    adata_vis_0 = mod.export_posterior(
        adata_vis_0, sample_kwargs={'num_samples': 3000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
    )
    mod.save(f"{run_name}", overwrite=True)

    # Compute expected expression per cell type
    expected_dict = mod.module.model.compute_expected_per_cell_type(
        mod.samples["post_sample_q05"], mod.adata_manager
    )

    # Add to anndata layers
    for i, n in enumerate(mod.factor_names_):
        adata_vis_0.layers[n] = expected_dict['mu'][i]

    # Save AnnData with results
    adata_file = f"{run_name}/sp.h5ad"
    adata_vis_0.write(adata_file)

    # Add cell abundance quantile to `obs`
    adata_vis_0.obs[adata_vis_0.uns['mod']['factor_names']] = adata_vis_0.obsm['q05_cell_abundance_w_sf']

    # Save proportions
    df = adata_vis_0.obsm['q95_cell_abundance_w_sf']
    total_abundance = df.sum(axis=1)
    proportions = df.div(total_abundance, axis=0)
    proportions.columns = [col.split('_')[-1] for col in proportions.columns]
    proportions.reset_index(inplace=True)
    proportions.rename(columns={'index': 'spot'}, inplace=True)
    proportions.to_csv(f"{run_name}/cell2loc_deconv_predictions.csv", index=False)

    # Export layers to CSV
    layers_output_dir = f"{run_name}/layers"
    os.makedirs(layers_output_dir, exist_ok=True)
    for layer_name in adata_vis_0.layers.keys():
        layer_data = adata_vis_0.layers[layer_name].toarray()
        df = pd.DataFrame(layer_data, index=adata_vis_0.obs.index, columns=adata_vis_0.var.index)
        df.to_csv(f"{layers_output_dir}/{layer_name}_layer.csv")

    # Generate and save plots
    fig_dir = f"{run_name}/plots"
    os.makedirs(fig_dir, exist_ok=True)

    # Extract a simplified replicate name
    simplified_name = os.path.splitext(os.path.basename(replicate_name))[0]

    # UMAP visualization
    sc.pl.embedding(
        adata_vis_0,
        basis="spatial",
        color=adata_vis_0.uns['mod']['factor_names'],
        cmap="magma",
        ncols=2,
        show=False,
        save=f"umap_{simplified_name}.pdf",
    )

    # Spatial abundance visualization
    for ct in adata_vis_0.uns['mod']['factor_names']:
        sc.pl.spatial(
            adata_vis_0,
            color=[ct],
            spot_size=1,
            cmap="magma",
            size=1.3,
            show=False,
            save=f"{simplified_name}_{ct}_abundance.pdf",
        )

    print(f"Figures and results saved for {simplified_name}")

/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/train/_trainrunner.py:76: UserWarning: `use_gpu` is deprecated in v1.0 and will be removed in v1.1. Please use `accelerator` and `devices` instead.
  accelerator, lightning_devices, device = parse_device_args(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/lightning/pytorch/loo

Epoch 30000/30000: 100%|██████████| 30000/30000 [31:10<00:00, 15.98it/s, v_num=1, elbo_train=1.98e+7]

`Trainer.fit` stopped: `max_epochs=30000` reached.


Epoch 30000/30000: 100%|██████████| 30000/30000 [31:10<00:00, 16.04it/s, v_num=1, elbo_train=1.98e+7]


/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/model/base/_pyromixin.py:388: UserWarning: `use_gpu` is deprecated in v1.0 and will be removed in v1.1. Please use `accelerator` and `devices` instead.
  _, _, device = parse_device_args(


...

Epoch 30000/30000: 100%|██████████| 30000/30000 [30:54<00:00, 15.80it/s, v_num=1, elbo_train=1.93e+7] 

`Trainer.fit` stopped: `max_epochs=30000` reached.


Epoch 30000/30000: 100%|██████████| 30000/30000 [30:54<00:00, 16.18it/s, v_num=1, elbo_train=1.93e+7]


/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/model/base/_pyromixin.py:388: UserWarning: `use_gpu` is deprecated in v1.0 and will be removed in v1.1. Please use `accelerator` and `devices` instead.
  _, _, device = parse_device_args(


Sampling local variables, batch:   0%|          | 0/1 [00:00<?, ?it/s]